# Experiment 009: Perturbation + Re-optimization

The baseline is at a local optimum. To escape it:
1. Perturb 15% of trees (random position shift, angle shift)
2. Run SA optimization to find new local optimum
3. If better, keep; if not, try different perturbation
4. Use population-based approach (keep top 3 solutions)

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import random
import math
import copy
import time

getcontext().prec = 25
scale_factor = Decimal("1e15")

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

print("Setup complete")

In [ ]:
class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def _update_polygon(self):
        vertices = []
        for tx, ty in zip(TX, TY):
            vertices.append((float(Decimal(str(tx)) * scale_factor), 
                           float(Decimal(str(ty)) * scale_factor)))
        initial_polygon = Polygon(vertices)
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor)
        )
    
    def set_params(self, x, y, angle):
        self.center_x = Decimal(str(x))
        self.center_y = Decimal(str(y))
        self.angle = Decimal(str(angle)) % 360
        self._update_polygon()
    
    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

def has_collision(trees):
    for i, t1 in enumerate(trees):
        for j, t2 in enumerate(trees):
            if i < j:
                if t1.polygon.intersects(t2.polygon) and not t1.polygon.touches(t2.polygon):
                    return True
    return False

def calculate_side(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return float(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / float(scale_factor)

def calculate_score(trees):
    n = len(trees)
    side = calculate_side(trees)
    return side**2 / n

print("Classes defined")

In [ ]:
def perturb_config(trees, position_delta=0.1, angle_delta=30, perturb_fraction=0.15, seed=None):
    """
    Perturb a configuration by randomly shifting positions and angles.
    
    Args:
        trees: List of ChristmasTree objects
        position_delta: Max position shift in each direction
        angle_delta: Max angle shift in degrees
        perturb_fraction: Fraction of trees to perturb (0.15 = 15%)
        seed: Random seed for reproducibility
    """
    if seed is not None:
        random.seed(seed)
    
    perturbed = [t.clone() for t in trees]
    n = len(perturbed)
    num_to_perturb = max(1, int(n * perturb_fraction))
    
    # Randomly select trees to perturb
    indices = random.sample(range(n), num_to_perturb)
    
    for i in indices:
        old_x = float(perturbed[i].center_x)
        old_y = float(perturbed[i].center_y)
        old_angle = float(perturbed[i].angle)
        
        new_x = old_x + random.uniform(-position_delta, position_delta)
        new_y = old_y + random.uniform(-position_delta, position_delta)
        new_angle = (old_angle + random.uniform(-angle_delta, angle_delta)) % 360
        
        perturbed[i].set_params(new_x, new_y, new_angle)
    
    return perturbed

print("Perturbation function defined")

In [ ]:
def simple_sa_optimize(trees, iterations=1000, T_start=0.01, T_end=0.0001, seed=None):
    """
    Simple simulated annealing optimization.
    """
    if seed is not None:
        random.seed(seed)
    
    current = [t.clone() for t in trees]
    current_score = calculate_score(current)
    
    best = [t.clone() for t in current]
    best_score = current_score
    
    T = T_start
    T_decay = (T_end / T_start) ** (1.0 / iterations)
    
    for it in range(iterations):
        # Pick a random tree
        i = random.randint(0, len(current) - 1)
        
        # Save old state
        old_x = float(current[i].center_x)
        old_y = float(current[i].center_y)
        old_angle = float(current[i].angle)
        
        # Random move
        move_type = random.choice(['translate', 'rotate'])
        if move_type == 'translate':
            delta = 0.01 * (T / T_start)  # Smaller moves as T decreases
            new_x = old_x + random.uniform(-delta, delta)
            new_y = old_y + random.uniform(-delta, delta)
            current[i].set_params(new_x, new_y, old_angle)
        else:
            delta = 5 * (T / T_start)  # Smaller rotations as T decreases
            new_angle = (old_angle + random.uniform(-delta, delta)) % 360
            current[i].set_params(old_x, old_y, new_angle)
        
        # Check collision
        if has_collision(current):
            current[i].set_params(old_x, old_y, old_angle)
            continue
        
        # Calculate new score
        new_score = calculate_score(current)
        delta_score = new_score - current_score
        
        # Accept or reject
        if delta_score < 0 or random.random() < math.exp(-delta_score / T):
            current_score = new_score
            if new_score < best_score:
                best_score = new_score
                best = [t.clone() for t in current]
        else:
            current[i].set_params(old_x, old_y, old_angle)
        
        T *= T_decay
    
    return best, best_score

print("SA optimizer defined")

In [ ]:
# Load baseline
baseline_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv')

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

print("Baseline loaded")

In [ ]:
# Test perturbation + SA on a few N values
test_ns = [50, 100, 150, 200]

results = {}
for n in test_ns:
    print(f"\n=== N={n} ===")
    baseline_trees = load_trees_for_n(baseline_df, n)
    baseline_score = calculate_score(baseline_trees)
    print(f"Baseline score: {baseline_score:.9f}")
    
    best_score = baseline_score
    best_trees = baseline_trees
    
    # Try multiple perturbations with different seeds
    for seed in range(10):
        # Perturb
        perturbed = perturb_config(baseline_trees, 
                                   position_delta=0.1, 
                                   angle_delta=30, 
                                   perturb_fraction=0.15,
                                   seed=seed)
        
        # Check if perturbed config has collisions
        if has_collision(perturbed):
            print(f"  Seed {seed}: Perturbed config has collisions, skipping")
            continue
        
        perturbed_score = calculate_score(perturbed)
        
        # Optimize
        optimized, opt_score = simple_sa_optimize(perturbed, iterations=500, seed=seed)
        
        print(f"  Seed {seed}: perturbed={perturbed_score:.9f} -> optimized={opt_score:.9f}")
        
        if opt_score < best_score:
            best_score = opt_score
            best_trees = optimized
            print(f"    *** NEW BEST! ***")
    
    improvement = baseline_score - best_score
    results[n] = {
        'baseline': baseline_score,
        'best': best_score,
        'improvement': improvement,
        'trees': best_trees
    }
    print(f"\nN={n} Summary: baseline={baseline_score:.9f}, best={best_score:.9f}, improvement={improvement:.9f}")

In [ ]:
# Summary
print("\n" + "="*60)
print("PERTURBATION + SA RESULTS")
print("="*60)

total_improvement = 0
for n, r in results.items():
    print(f"N={n}: {r['baseline']:.9f} -> {r['best']:.9f} (improvement: {r['improvement']:.9f})")
    total_improvement += r['improvement']

print(f"\nTotal improvement on tested N values: {total_improvement:.9f}")

if total_improvement > 0:
    print("\n*** IMPROVEMENTS FOUND! ***")
else:
    print("\nNo improvements found. The baseline is at a very tight local optimum.")

In [ ]:
# Copy baseline to submission (since likely no improvements)
import shutil
shutil.copy('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv',
            '/home/submission/submission.csv')
print("Copied baseline to submission")